# Imports

In [14]:
%reload_ext autoreload
%autoreload 2

In [15]:
import pickle
import os
from time import time
import numpy as np

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, MaxPooling1D, Concatenate
from tensorflow.keras.layers import Dense, Conv1D, Dropout, BatchNormalization, LeakyReLU, ELU
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Softmax

# Data loading

In [16]:
n_tips = ['674', '489', '87']

In [17]:
pickle_base = '/workspace/deep_birth_death/deep_learning/pickles/testing_k/dataset_'
data = dict()
for i in n_tips:
    with open(pickle_base + i + "_10k.pkl", 'rb') as f:
        data[i] = pickle.load(f)

# Model definition

In [18]:
def create_nn(n_out, n_tips, clas=False, div_scenario = None):
    input_data = Input(shape=(n_tips, 1))

    final_filters = 128
    x = Conv1D(16, kernel_size=3, padding='same')(input_data)
    x = ELU()(x)
    x = Conv1D(16, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    if n_tips > 256:
        final_filters = 64
        x = Conv1D(32, kernel_size=3, padding='same')(x)
        x = ELU()(x)
        x = Conv1D(32, kernel_size=3, padding='same')(x)
        x = ELU()(x)
        x = MaxPooling1D(pool_size=2)(x)
    
        if n_tips > 512:
            final_filters = 128
            x = Conv1D(64, kernel_size=3, padding='same')(x)
            x = ELU()(x)
            x = Conv1D(64, kernel_size=3, padding='same')(x)
            x = ELU()(x)
            x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(final_filters, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)

    x = GlobalAveragePooling1D()(x)

    x = Dense(32)(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(n_out, name='logits')(x)
    if clas:
        
        output_class = Softmax()(x)
        
    else:
        if div_scenario != "SAT":
            out_list = []

            for i in range(n_out):
                y = Dense(32)(x)
                y = ELU()(y)
                y = Dropout(0.3)(y)

                y = Dense(1)(y)
                y = LeakyReLU(alpha=10)(y)
                out_list.append(y)

            output_class = Concatenate()(out_list)
            
        elif div_scenario == "WW":
            out_list = []

            for i in range(n_out):
                y = Dense(32)(x)
                y = ELU()(y)
                y = Dropout(0.3)(y)

                y = Dense(1)(y)
                y = Linear(y)
                out_list.append(y)

            output_class = Concatenate()(out_list)                
            
        else:
            x = Dense(32)(x)
            x = ELU()(x)
            x = Dropout(0.3)(x)
            x = Dense(1)(x)
            output_class = LeakyReLU(alpha=10)(x)
            
    return Model(input_data, output_class)

# Training

In [19]:
callback = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
out_folder_path = "/workspace/deep_birth_death/deep_learning/models/"
os.makedirs(out_folder_path + 'class/', exist_ok=True)
os.makedirs(out_folder_path + 'reg/', exist_ok=True)

## Classification training

In [ ]:
for i in n_tips:
    print("\nClasification training", i, 'tips')

    nn_model = create_nn(len(data[i]['y_class_train'][0]),
                         int(i), clas=True)
    nn_model.compile(loss="categorical_crossentropy",
                     optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    start = time()
    history = nn_model.fit(data[i]['X_train'], data[i]['y_class_train'],
                           batch_size=128, epochs=1000, validation_split=0.1,
                           callbacks=[callback], verbose=1)
    elapsed_time = time()-start
    print('Elapsed time', elapsed_time)

    save_path = out_folder_path + 'testing_k_class/' + i + "_classification_temperature"

    nn_model.save(save_path + "model.keras")
    with open(save_path + "history.pkl", 'wb') as f:
            pickle.dump(history.history, f)
    with open(save_path + "model_data.pkl", 'wb') as f:
            pickle.dump([nn_model.count_params(), elapsed_time], f)


Clasification training 674 tips
Epoch 1/1000
380/380 [==============================] - 4s 9ms/step - loss: 0.8515 - accuracy: 0.6519 - val_loss: 0.5965 - val_accuracy: 0.7537
Epoch 2/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.5395 - accuracy: 0.7885 - val_loss: 0.5042 - val_accuracy: 0.7983
Epoch 3/1000
380/380 [==============================] - 4s 9ms/step - loss: 0.4835 - accuracy: 0.8134 - val_loss: 0.4308 - val_accuracy: 0.8356
Epoch 4/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.4599 - accuracy: 0.8216 - val_loss: 0.4076 - val_accuracy: 0.8430
Epoch 5/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.4441 - accuracy: 0.8271 - val_loss: 0.4096 - val_accuracy: 0.8372
Epoch 6/1000
380/380 [==============================] - 4s 9ms/step - loss: 0.4349 - accuracy: 0.8301 - val_loss: 0.3913 - val_accuracy: 0.8493
Epoch 7/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.4226 - accuracy: 0.83

Epoch 57/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2673 - accuracy: 0.8977 - val_loss: 0.2574 - val_accuracy: 0.9067
Epoch 58/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2646 - accuracy: 0.8994 - val_loss: 0.2468 - val_accuracy: 0.9089
Epoch 59/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2642 - accuracy: 0.9004 - val_loss: 0.2421 - val_accuracy: 0.9100
Epoch 60/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2647 - accuracy: 0.9005 - val_loss: 0.2430 - val_accuracy: 0.9096
Epoch 61/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2608 - accuracy: 0.9015 - val_loss: 0.2396 - val_accuracy: 0.9102
Epoch 62/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2599 - accuracy: 0.9034 - val_loss: 0.2501 - val_accuracy: 0.9094
Epoch 63/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2596 - accuracy: 0.9029 - val_loss: 0.2428 -

380/380 [==============================] - 4s 9ms/step - loss: 0.2219 - accuracy: 0.9176 - val_loss: 0.2098 - val_accuracy: 0.9248
Epoch 114/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2234 - accuracy: 0.9173 - val_loss: 0.2119 - val_accuracy: 0.9213
Epoch 115/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2218 - accuracy: 0.9185 - val_loss: 0.2511 - val_accuracy: 0.9028
Epoch 116/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2220 - accuracy: 0.9170 - val_loss: 0.2108 - val_accuracy: 0.9252
Epoch 117/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2234 - accuracy: 0.9175 - val_loss: 0.2302 - val_accuracy: 0.9119
Epoch 118/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2194 - accuracy: 0.9201 - val_loss: 0.2127 - val_accuracy: 0.9213
Epoch 119/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2210 - accuracy: 0.9188 - val_loss: 0.2074 - val_accu

Epoch 169/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2058 - accuracy: 0.9250 - val_loss: 0.2155 - val_accuracy: 0.9211
Epoch 170/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2079 - accuracy: 0.9235 - val_loss: 0.2064 - val_accuracy: 0.9220
Epoch 171/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2034 - accuracy: 0.9257 - val_loss: 0.2049 - val_accuracy: 0.9267
Epoch 172/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2030 - accuracy: 0.9252 - val_loss: 0.2002 - val_accuracy: 0.9285
Epoch 173/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2021 - accuracy: 0.9261 - val_loss: 0.2022 - val_accuracy: 0.9246
Epoch 174/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2025 - accuracy: 0.9254 - val_loss: 0.1949 - val_accuracy: 0.9265
Epoch 175/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.2050 - accuracy: 0.9242 - val_loss: 0

Epoch 225/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1929 - accuracy: 0.9289 - val_loss: 0.2004 - val_accuracy: 0.9263
Epoch 226/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1927 - accuracy: 0.9293 - val_loss: 0.1920 - val_accuracy: 0.9298
Epoch 227/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1904 - accuracy: 0.9298 - val_loss: 0.2104 - val_accuracy: 0.9250
Epoch 228/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1892 - accuracy: 0.9303 - val_loss: 0.1917 - val_accuracy: 0.9307
Epoch 229/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1907 - accuracy: 0.9288 - val_loss: 0.1914 - val_accuracy: 0.9311
Epoch 230/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1912 - accuracy: 0.9312 - val_loss: 0.1922 - val_accuracy: 0.9287
Epoch 231/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1912 - accuracy: 0.9304 - val_loss: 0

Epoch 281/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1807 - accuracy: 0.9333 - val_loss: 0.2032 - val_accuracy: 0.9248
Epoch 282/1000
380/380 [==============================] - 4s 10ms/step - loss: 0.1835 - accuracy: 0.9317 - val_loss: 0.1952 - val_accuracy: 0.9287
Epoch 283/1000
380/380 [==============================] - 3s 8ms/step - loss: 0.1824 - accuracy: 0.9323 - val_loss: 0.1943 - val_accuracy: 0.9276
Epoch 284/1000
380/380 [==============================] - 2s 6ms/step - loss: 0.1769 - accuracy: 0.9350 - val_loss: 0.1907 - val_accuracy: 0.9306
Epoch 285/1000
380/380 [==============================] - 2s 6ms/step - loss: 0.1793 - accuracy: 0.9336 - val_loss: 0.1874 - val_accuracy: 0.9304
Epoch 286/1000
380/380 [==============================] - 3s 7ms/step - loss: 0.1786 - accuracy: 0.9337 - val_loss: 0.1954 - val_accuracy: 0.9283
Epoch 287/1000
380/380 [==============================] - 3s 9ms/step - loss: 0.1789 - accuracy: 0.9331 - val_loss: 0.1917

## Regression training

In [ ]:
for i in n_tips:
    for label in np.unique(data[i]['div_info_train']):
        div_scenario = label.split('/')[1].split('_')[0]
        for data_norm in ['_norm']:

            # Get regression values of the corresponding scenario
            X_train = data[i]['X_train'][data[i]['div_info_train'] == label]
            y_reg_train = data[i]['y_reg' + data_norm + '_train'][data[i]['div_info_train'] == label]
            y_reg_train = [np.array(elem) for elem in y_reg_train]
            
            resc_factor_train = data[i]['resc_factor_train'][data[i]['div_info_train'] == label]
            
            print("\nRegression training", i, 'tips,', div_scenario, 'model,', data_norm, 'norm')
            nn_model = create_nn(len(y_reg_train[0]),
                                 int(i), div_scenario=div_scenario)
            nn_model.compile(loss="mae", optimizer=Adam(learning_rate=0.001),
                             metrics=['mse'])

            start = time()
            history = nn_model.fit(np.expand_dims(X_train, axis=2),
                                   np.expand_dims(y_reg_train, axis=2),
                                   batch_size=128, epochs=1000, validation_split=0.1,
                                   callbacks=[callback], verbose=0)
            elapsed_time = time()-start
            print('Elapsed time', elapsed_time)

            save_path = out_folder_path + 'reg/' + div_scenario + '/'
            os.makedirs(save_path, exist_ok=True)
            save_path +=  i + "_regression" + data_norm + '_'

            nn_model.save(save_path + "model.keras")
            with open(save_path + "history.pkl", 'wb') as f:
                    pickle.dump(history.history, f)
            with open(save_path + "model_data.pkl", 'wb') as f:
                    pickle.dump([nn_model.count_params(), elapsed_time], f)
                    
            pred = nn_model.predict(np.expand_dims(data[i]['X_test'][data[i]['div_info_test'] == label], axis=2))
            y_reg_test = data[i]['y_reg' + data_norm + '_test'][data[i]['div_info_test'] == label]
            y_reg_test = [np.array(elem) for elem in y_reg_test]
            resc_factor_test = data[i]['resc_factor_test'][data[i]['div_info_test'] == label]

            real = y_reg_test
            error = (pred-real)**2
            
            print('Errors')
            for j in range(len(pred[0])):
                print(np.mean(error[:,j]))
            print('Total error:', np.mean(error))
            
            pred/=resc_factor_test[:, np.newaxis]
            real = y_reg_test
            real/=resc_factor_test[:, np.newaxis]
            error = (pred-real)**2
            print('Denorm Errors')
            for j in range(len(pred[0])):
                print(np.mean(error[:,j]))
            print('Total error:', np.mean(error))